In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 0


#  Seed 
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

#  TF determinism
os.environ["TF_DETERMINISTIC_OPS"] = "1"

# limits parallelism to reduce non-determinism
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

print("Seeds set to", SEED)

Seeds set to 0


# Replica

In this section, we attempt to replicate the methodology proposed in the original paper.

### 1. Input construction

In [ ]:
def load_asset_csv_price(path, asset_name):
    """
    CSV with columns:
    date | price | open | high | low | volume | change%
    """
    df = pd.read_csv(path)

    df.columns = [c.strip().lower() for c in df.columns]

    # Detect date column
    if "date" in df.columns:
        date_col = "date"
    else:
        raise ValueError(f"Colonna 'date' non trovata in {path}")

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()

    if "price" not in df.columns:
        raise ValueError(f"Colonna 'price' non trovata in {path}")

    price = df["price"].astype(float)
    price.name = asset_name
    return price

prices = pd.concat(
    [
        load_asset_csv_price("VTI 2006-2020.csv", "VTI"),
        load_asset_csv_price("AGG 2006-2020.csv", "AGG"),
        load_asset_csv_price("DBC 2006-2020.csv", "DBC"),
        load_asset_csv_price("VIX 2006-2020.csv", "VIX"),
    ],
    axis=1
)

print(prices.isna().sum())

VTI     6
AGG     5
DBC    32
VIX    26
dtype: int64


In [23]:
# prices_raw = DataFrame with columns ['VTI','AGG','DBC','VIX']
prices_raw = prices.copy()

# VTI calendar as reference
calendar = prices_raw["VTI"].dropna().index
prices_aligned = prices_raw.reindex(calendar)

# forward-fill 
prices_aligned = prices_aligned.ffill()

# drop NaN 
prices = prices_aligned.dropna()

# filter date range
prices = prices.loc["2006-01-01":"2020-04-30"]

print("NaN residuals:\n", prices.isna().sum())
print("Final Shape:", prices.shape)
print(prices.head())
print(prices.tail())

NaN residuals:
 VTI    0
AGG    0
DBC    0
VIX    0
dtype: int64
Final Shape: (3587, 4)
              VTI     AGG    DBC    VIX
date                                   
2006-02-06  63.16  100.21  24.20  13.04
2006-02-07  62.55  100.14  23.50  13.59
2006-02-08  63.00  100.09  23.40  12.83
2006-02-09  62.88  100.15  23.62  13.12
2006-02-10  63.01   99.94  23.19  12.87
               VTI     AGG    DBC    VIX
date                                    
2020-04-24  141.63  117.36  10.70  35.93
2020-04-27  144.14  116.82  10.50  33.29
2020-04-28  143.69  117.28  10.51  33.57
2020-04-29  147.76  117.36  10.70  31.23
2020-04-30  145.84  117.10  10.90  34.15


In [4]:
def make_features_and_labels(prices: pd.DataFrame, lookback=50):
    prices = prices.sort_index()
    rets = prices.pct_change().dropna()
    prices = prices.loc[rets.index]

    X_day = np.concatenate([prices.values, rets.values], axis=1)  # (T, 2*A)
    A = prices.shape[1]
    dates = rets.index

    X, Y, D = [], [], []
    for t in range(lookback, len(rets)):
        X.append(X_day[t - lookback : t, :])      # (lookback, 2*A)
        Y.append(rets.values[t, :])               # (A,)
        D.append(dates[t])                        # date of r_t
    return np.asarray(X, np.float32), np.asarray(Y, np.float32), pd.to_datetime(D), A

def idx_between(dates, start, end):
    start = pd.to_datetime(start)
    end   = pd.to_datetime(end)
    return np.where((dates >= start) & (dates <= end))[0]


### 2. Differentiable EWMA

In [5]:
@tf.function
def ewma_sigma_from_returns_window(r_window, span=50, eps=1e-8):
    lam = 1.0 - 2.0 / (span + 1.0)
    r2 = tf.square(r_window)                       # (N,k,A)
    v0 = tf.reduce_mean(r2, axis=1)                # (N,A)
    r2_T = tf.transpose(r2, perm=[1, 0, 2])        # (k,N,A)

    def step(v_prev, r2_t):
        return lam * v_prev + (1.0 - lam) * r2_t

    v_T = tf.scan(step, r2_T, initializer=v0)      # (k,N,A)
    v_last = v_T[-1]                               # (N,A)
    return tf.sqrt(v_last + eps)

@tf.function
def sharpe_ratio(x, eps=1e-8):
    mu = tf.reduce_mean(x)
    sd = tf.math.reduce_std(x) + eps
    return mu / sd

### 3. Helper for computing scaled and turnover returns

In [6]:
@tf.function
def modified_return_eq7_sequence(x, r_t, w_tm1, n_assets, sigma_tgt, C=1e-4, eps=1e-8, use_scaling=True):
    r_win = x[:, :, n_assets:2*n_assets]                                 # (N,50,A)
    sigma_tm1 = ewma_sigma_from_returns_window(r_win, span=50, eps=eps)   # (N,A)

    w_tm2 = tf.concat([w_tm1[:1], w_tm1[:-1]], axis=0)                    # (N,A)
    sigma_tm2 = tf.concat([sigma_tm1[:1], sigma_tm1[:-1]], axis=0)        # (N,A)

    if use_scaling:
        s_tgt = tf.cast(sigma_tgt, tf.float32)
        scale_tm1 = s_tgt / (sigma_tm1 + eps)
        scale_tm2 = s_tgt / (sigma_tm2 + eps)

        wS_tm1 = scale_tm1 * w_tm1
        wS_tm2 = scale_tm2 * w_tm2
    else:
        # no volatility scaling: use raw weights
        wS_tm1 = w_tm1
        wS_tm2 = w_tm2

    gross = tf.reduce_sum(wS_tm1 * r_t, axis=1)                           # (N,)
    turnover = tf.reduce_sum(tf.abs(wS_tm1 - wS_tm2), axis=1)             # (N,)
    cost = tf.cast(C, tf.float32) * turnover
    return gross - cost, turnover, cost

### 4. LSTM Portfolio

In [ ]:
class LSTMPortfolioEq7(tf.keras.Model):
    def __init__(self, n_assets, lstm_units=64, sigma_tgt=0.10, C=1e-4,
                 use_scaling=True, train_on_eq7=True):
        super().__init__()
        self.n_assets = int(n_assets)
        self.sigma_tgt = float(sigma_tgt)
        self.C = float(C)
        self.use_scaling = bool(use_scaling)
        self.train_on_eq7 = bool(train_on_eq7)

        self.lstm = tf.keras.layers.LSTM(lstm_units)
        self.out  = tf.keras.layers.Dense(self.n_assets, activation="softmax")

    def call(self, x, training=False):
        h = self.lstm(x, training=training)
        return self.out(h)

    def train_step(self, data):
        x, r_t = data
        with tf.GradientTape() as tape:
            w_tm1 = self(x, training=True)

            if self.train_on_eq7:
                Rp, turnover, cost = modified_return_eq7_sequence(
                    x, r_t, w_tm1,
                    n_assets=self.n_assets,
                    sigma_tgt=self.sigma_tgt,
                    C=self.C,
                    use_scaling=self.use_scaling
                )
            else:
                # GROSS return: no scaling, no costs
                Rp = tf.reduce_sum(w_tm1 * r_t, axis=1)  # (batch,)
                turnover = tf.zeros_like(Rp)
                cost = tf.zeros_like(Rp)

            J = sharpe_ratio(Rp)
            loss = -J

        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {
            "loss": loss,
            "sharpe_obj": J,
            "avg_turnover": tf.reduce_mean(turnover),
            "avg_cost": tf.reduce_mean(cost),
        }

    def test_step(self, data):
        x, r_t = data
        w_tm1 = self(x, training=False)

        Rp_mod, turnover, cost = modified_return_eq7_sequence(
            x, r_t, w_tm1,
            n_assets=self.n_assets,
            sigma_tgt=self.sigma_tgt,
            C=self.C,
            use_scaling=self.use_scaling
        )

        J = sharpe_ratio(Rp_mod)
        loss = -J
        return {
            "loss": loss,
            "sharpe_obj": J,
            "avg_turnover": tf.reduce_mean(turnover),
            "avg_cost": tf.reduce_mean(cost),
        }
    
def fit_on_period_with_val(
    X_train_full, Y_train_full, n_assets,
    lstm_units=64, sigma_tgt=0.10, C=1e-4,
    lr=1e-3, epochs=100, val_frac=0.10,
    use_early_stopping=False,
    use_scaling=True,
    train_on_eq7=True,
    batch_size=64
):
    N = len(X_train_full)
    n_val = max(1, int(np.floor(val_frac * N)))
    if N - n_val < 50:
        raise ValueError(f"Training too short after split val: N={N}, n_val={n_val}")

    # temporal split: train = initial part, val = last 10%
    X_tr, Y_tr = X_train_full[:-n_val], Y_train_full[:-n_val]
    X_va, Y_va = X_train_full[-n_val:], Y_train_full[-n_val:]

    model = LSTMPortfolioEq7(
        n_assets=n_assets,
        lstm_units=lstm_units,
        sigma_tgt=sigma_tgt,
        C=C,
        use_scaling=use_scaling,
        train_on_eq7=train_on_eq7
    )
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), run_eagerly=False)

    # mini-batch time-ordered (NO shuffle)
    ds_tr = tf.data.Dataset.from_tensor_slices((X_tr, Y_tr)).batch(batch_size, drop_remainder=False)
    ds_va = tf.data.Dataset.from_tensor_slices((X_va, Y_va)).batch(batch_size, drop_remainder=False)

    callbacks = []
    if use_early_stopping:
        callbacks.append(tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10, restore_best_weights=True
        ))

    hist = model.fit(ds_tr, validation_data=ds_va, epochs=epochs, verbose=0, callbacks=callbacks)
    return model, hist

### 5. Backtesting helper

In [8]:
def backtest_period(model, X_period, Y_period, n_assets, sigma_tgt=0.10, C=1e-4, use_scaling=True):
    x = tf.convert_to_tensor(X_period, tf.float32)
    r = tf.convert_to_tensor(Y_period, tf.float32)
    w = model(x, training=False)

    Rp_mod, turnover, cost = modified_return_eq7_sequence(
        x, r, w,
        n_assets=n_assets,
        sigma_tgt=sigma_tgt,
        C=C,
        use_scaling=use_scaling,
    )
    return w.numpy(), Rp_mod.numpy(), turnover.numpy(), cost.numpy()


### 6. Final wrapper

In [ ]:
def run_biennial_retraining_with_val(
    prices: pd.DataFrame,
    lookback=50,
    lstm_units=64,
    sigma_tgt=0.10,
    C_train=1e-4,
    C_eval=1e-4,
    lr=1e-3,
    epochs=100,
    val_frac=0.10,
    train_start="2006-01-01",
    first_train_end="2010-12-31",
    test_start="2011-01-01",
    test_end="2020-04-30",
    step_years=2,
    use_early_stopping=False,
    use_scaling=True,
    train_on_eq7=True,
    batch_size=64
):
    X, Y, dates, A = make_features_and_labels(prices, lookback=lookback)

    test_start = pd.to_datetime(test_start)
    test_end   = pd.to_datetime(test_end)

    if train_on_eq7:
        C_train_used = C_train 
    else:
        C_train_used = 0.0  # gross Sharpe 

    all_weights, all_Rp, all_turnover, all_cost, all_dates = [], [], [], [], []
    train_end = pd.to_datetime(first_train_end)
    current_test_start = pd.to_datetime(test_start)

    while current_test_start <= test_end:
        current_test_end = min(
            current_test_start + pd.DateOffset(years=step_years) - pd.DateOffset(days=1),
            test_end
        )

        # TRAIN expanding
        tr_idx = idx_between(dates, train_start, train_end)
        if len(tr_idx) < 100:
            raise ValueError(f"Insufficient training data between {train_start} and {train_end}: {len(tr_idx)}")

        model, hist = fit_on_period_with_val(
            X[tr_idx], Y[tr_idx], n_assets=A,
            lstm_units=lstm_units,
            sigma_tgt=sigma_tgt,
            C=C_train_used,             
            lr=lr, epochs=epochs, val_frac=val_frac,
            use_early_stopping=use_early_stopping,
            use_scaling=use_scaling,
            train_on_eq7=train_on_eq7,
            batch_size=batch_size
        )

        # TEST
        te_idx = idx_between(dates, current_test_start, current_test_end)
        if len(te_idx) == 0:
            break

        w, Rp_mod, turnover, cost = backtest_period(
            model, X[te_idx], Y[te_idx], n_assets=A,
            sigma_tgt=sigma_tgt,
            C=C_eval,                    # evaluation cost always used
            use_scaling=use_scaling,
        )

        all_weights.append(w)
        all_Rp.append(Rp_mod)
        all_turnover.append(turnover)
        all_cost.append(cost)
        all_dates.append(dates[te_idx])

        # update expanding boundary
        train_end = current_test_end
        current_test_start = current_test_end + pd.DateOffset(days=1)

    W = np.vstack(all_weights)
    Rp = np.concatenate(all_Rp)
    TO = np.concatenate(all_turnover)
    CO = np.concatenate(all_cost)
    D  = pd.to_datetime(np.concatenate(all_dates))

    out = pd.DataFrame({"Rp_mod": Rp, "turnover": TO, "cost": CO}, index=D).sort_index()
    out["cum_return"] = (1.0 + out["Rp_mod"]).cumprod()

    
    out.attrs["train_on_eq7"] = bool(train_on_eq7)
    out.attrs["use_scaling"] = bool(use_scaling)
    out.attrs["sigma_tgt"] = float(sigma_tgt)
    out.attrs["C_train_used"] = float(C_train_used)
    out.attrs["C_eval"] = float(C_eval)

    return W, out

### 7. Backtesting metrics

In [ ]:
TRADING_DAYS = 252

def max_drawdown_from_returns(r: pd.Series) -> float:
    cum = (1.0 + r).cumprod()
    dd = cum / cum.cummax() - 1.0
    return float(dd.min())  

def paper_metrics_from_returns(r: pd.Series, periods_per_year=TRADING_DAYS) -> dict:
    r = r.dropna().astype(float)
    if len(r) < 10:
        raise ValueError("Series too short for robust metrics.")

    mu_d = r.mean()
    sd_d = r.std(ddof=1)

    # annualization
    E_R   = mu_d * periods_per_year
    Std_R = sd_d * np.sqrt(periods_per_year)

    sharpe = (mu_d / (sd_d + 1e-12)) * np.sqrt(periods_per_year)

    # downside deviation
    neg = r[r < 0]
    if len(neg) > 1:
        dd_d = neg.std(ddof=1)
    else:
        dd_d = 0.0
    DD_R = dd_d * np.sqrt(periods_per_year)

    sortino = (mu_d / (dd_d + 1e-12)) * np.sqrt(periods_per_year) if dd_d > 0 else np.nan

    mdd = max_drawdown_from_returns(r)

    pct_pos = float((r > 0).mean())

    pos = r[r > 0]
    ave_p = pos.mean() if len(pos) > 0 else np.nan
    ave_l = neg.mean() if len(neg) > 0 else np.nan  
    ave_p_over_ave_l = float(ave_p / (abs(ave_l) + 1e-12)) if (len(pos) > 0 and len(neg) > 0) else np.nan

    return {
        "E(R) (ann.)": E_R,
        "Std(R) (ann.)": Std_R,
        "Sharpe (ann.)": sharpe,
        "DD(R) (ann.)": DD_R,
        "Sortino (ann.)": sortino,
        "MDD": mdd,
        "% of + Ret": pct_pos,
        "Ave. P / Ave. L": ave_p_over_ave_l,
    }

def backtest_report(perf: pd.DataFrame, periods_per_year=TRADING_DAYS,
                    by_blocks: bool = True) -> pd.DataFrame:

    if "Rp_mod" not in perf.columns:
        raise ValueError("perf must contain the column 'Rp_mod'.")

    perf = perf.sort_index()

    rows = []
    m_all = paper_metrics_from_returns(perf["Rp_mod"], periods_per_year=periods_per_year)
    if "turnover" in perf.columns:
        m_all["Avg Turnover"] = float(perf["turnover"].mean())
    if "cost" in perf.columns:
        m_all["Avg Cost"] = float(perf["cost"].mean())
    m_all["Start"] = perf.index.min()
    m_all["End"]   = perf.index.max()
    m_all["N"]     = int(perf["Rp_mod"].dropna().shape[0])
    rows.append(("ALL", m_all))

    # For biennial blocks (consistent with retraining every 2 years)
    if by_blocks:
        start_year = int(perf.index.min().year)
        end_year   = int(perf.index.max().year)

        # pairing blocks: 2011-2012, 2013-2014, ...
        y = start_year
        while y <= end_year:
            block_start = pd.Timestamp(year=y, month=1, day=1)
            block_end   = pd.Timestamp(year=min(y+1, end_year), month=12, day=31)

            block = perf.loc[(perf.index >= block_start) & (perf.index <= block_end)]
            if len(block) >= 10:
                m = paper_metrics_from_returns(block["Rp_mod"], periods_per_year=periods_per_year)
                if "turnover" in block.columns:
                    m["Avg Turnover"] = float(block["turnover"].mean())
                if "cost" in block.columns:
                    m["Avg Cost"] = float(block["cost"].mean())
                m["Start"] = block.index.min()
                m["End"]   = block.index.max()
                m["N"]     = int(block["Rp_mod"].dropna().shape[0])
                rows.append((f"{y}-{y+1}", m))
            y += 2

    report = pd.DataFrame({name: vals for name, vals in rows}).T

   
    ordered_cols = [
        "E(R) (ann.)","Std(R) (ann.)","Sharpe (ann.)","DD(R) (ann.)","Sortino (ann.)",
        "MDD","% of + Ret","Ave. P / Ave. L",
        "Avg Turnover","Avg Cost","N","Start","End"
    ]
    existing = [c for c in ordered_cols if c in report.columns]
    report = report[existing]
    return report

### 8. Evaluation

We compare alternative training objectives for the LSTM and find that the Sharpe-ratio–based loss that incorporates
 volatility targeting and turnover costs, provides a superior training signal and leads to more stable and robust portfolio performance.

In [11]:
SIGMA_DAILY_10 = 0.10 / np.sqrt(252.0)

scenarios = [
    ("No scaling, C=0",        False, SIGMA_DAILY_10, 0.0),
    ("Scaling 10% ann, C=1bp", True,  SIGMA_DAILY_10, 1e-4),
    ("Scaling 10% ann, C=10bp",True,  SIGMA_DAILY_10, 1e-3),
]

all_reports = []
for name, use_scaling, sigma_tgt, C in scenarios:
    tf.keras.backend.clear_session()
    W, perf = run_biennial_retraining_with_val(
    prices=prices,
    lookback=50,
    lstm_units=64,
    sigma_tgt=sigma_tgt,
    C_train=1e-4,    
    C_eval=C,        
    lr=1e-3,
    epochs=100,
    val_frac=0.10,
    use_scaling=use_scaling,
    train_on_eq7=False   
)
    rep = backtest_report(perf, by_blocks=False)
    rep.insert(0, "scenario", name)
    rep.insert(1, "use_scaling", use_scaling)
    rep.insert(2, "sigma_tgt_daily", sigma_tgt)
    rep.insert(3, "C", C)
    all_reports.append(rep)

summary = pd.concat(all_reports, axis=0).reset_index(drop=True)
summary

I0000 00:00:1767090290.376753 2908843 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1767090290.376803 2908843 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


,scenario,use_scaling,sigma_tgt_daily,C,E(R) (ann.),Std(R) (ann.),Sharpe (ann.),DD(R) (ann.),Sortino (ann.),MDD,% of + Ret,Ave. P / Ave. L,Avg Turnover,Avg Cost,N,Start,End
0,"No scaling, C=0",False,0.006299,0.0000,0.271105,0.191401,1.416423,0.106622,2.542664,-0.106671,0.541046,1.267893,0.061468,0.0,2351,2011-01-03 00:00:00,2020-04-30 00:00:00
1,"Scaling 10% ann, C=1bp",True,0.006299,0.0001,0.084522,0.067197,1.257831,0.051745,1.633428,-0.099651,0.548703,1.027695,0.101364,0.00001,2351,2011-01-03 00:00:00,2020-04-30 00:00:00
2,"Scaling 10% ann, C=10bp",True,0.006299,0.0010,0.052629,0.067839,0.775787,0.053931,0.975855,-0.146575,0.544875,0.957161,0.09596,0.000096,2351,2011-01-03 00:00:00,2020-04-30 00:00:00


In [12]:
all_reports = []
for name, use_scaling, sigma_tgt, C in scenarios:
    tf.keras.backend.clear_session()
    W, perf = run_biennial_retraining_with_val(
    prices=prices,
    lookback=50,
    lstm_units=64,
    sigma_tgt=sigma_tgt,
    C_train=1e-4,    
    C_eval=C,        
    lr=1e-3,
    epochs=100,
    val_frac=0.10,
    use_scaling=use_scaling,
    train_on_eq7=True  
)
    rep = backtest_report(perf, by_blocks=False)
    rep.insert(0, "scenario", name)
    rep.insert(1, "use_scaling", use_scaling)
    rep.insert(2, "sigma_tgt_daily", sigma_tgt)
    rep.insert(3, "C", C)
    all_reports.append(rep)

summary = pd.concat(all_reports, axis=0).reset_index(drop=True)
summary

,scenario,use_scaling,sigma_tgt_daily,C,E(R) (ann.),Std(R) (ann.),Sharpe (ann.),DD(R) (ann.),Sortino (ann.),MDD,% of + Ret,Ave. P / Ave. L,Avg Turnover,Avg Cost,N,Start,End
0,"No scaling, C=0",False,0.006299,0.0000,0.233429,0.13363,1.746829,0.072065,3.239141,-0.096617,0.553382,1.23777,0.04491,0.0,2351,2011-01-03 00:00:00,2020-04-30 00:00:00
1,"Scaling 10% ann, C=1bp",True,0.006299,0.0001,0.105332,0.060117,1.752095,0.036633,2.875279,-0.09093,0.544875,1.22712,0.031419,0.000003,2351,2011-01-03 00:00:00,2020-04-30 00:00:00
2,"Scaling 10% ann, C=10bp",True,0.006299,0.0010,0.08413,0.061178,1.375176,0.03596,2.339521,-0.079772,0.529137,1.188374,0.025769,0.000026,2351,2011-01-03 00:00:00,2020-04-30 00:00:00


When training is performed using a Sharpe-ratio–based objective that incorporates volatility targeting and transaction costs, the resulting performance closely matches the Deep Learning Strategy (DLS) reported in the original paper. In particular, for realistic transaction cost levels, the replicated strategy achieves comparable Sharpe ratios, confirming the validity of the proposed training framework.

It is important to note, however, that the training process is inherently noisy and exhibits a non-negligible dependence on the random initialization (seed). As a result, individual training runs may display variability in out-of-sample performance, even when the model architecture and hyperparameters are held fixed. This behavior is consistent with the non-convex optimization landscape of deep learning models and further motivates the use of repeated runs or robust evaluation protocols when assessing performance.

# Verification

In this section, we aim to verify several claims made in the paper regarding feature engineering, network architecture, and the role of cross-validation in model performance.

In [ ]:
# ============================================================
# 1) Load OHLCV per asset
# ============================================================
def load_asset_csv_full(path: str, asset_name: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = [c.strip().lower() for c in df.columns]

    if "date" not in df.columns:
        raise ValueError(f"Colonna 'date' non trovata in {path}")

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").set_index("date")

    out = pd.DataFrame(index=df.index)
    for col in ["price", "open", "high", "low", "volume"]:
        out[col] = pd.to_numeric(df[col], errors="coerce") if col in df.columns else np.nan

    out = out.rename(columns={"price": "close"})
    out.columns = [f"{asset_name}__{c}" for c in out.columns]
    return out


# ============================================================
# 2) STATIC FEATURES (NO rolling)
# ============================================================
def build_static_features_from_ohlcv(df_asset: pd.DataFrame, asset_name: str) -> pd.DataFrame:
    c = f"{asset_name}__close"
    o = f"{asset_name}__open"
    h = f"{asset_name}__high"
    l = f"{asset_name}__low"
    v = f"{asset_name}__volume"

    out = pd.DataFrame(index=df_asset.index)
    close = df_asset[c].astype(float)

    # price action (no returns)
    if h in df_asset.columns and l in df_asset.columns:
        high = df_asset[h].astype(float)
        low  = df_asset[l].astype(float)
        out[f"{asset_name}__hl_range"] = (high - low) / close
        out[f"{asset_name}__hl_pos"]   = (close - low) / (high - low + 1e-8)

    if o in df_asset.columns:
        open_ = df_asset[o].astype(float)
        out[f"{asset_name}__co_ret"] = (close - open_) / open_

    # volume
    if v in df_asset.columns and not df_asset[v].isna().all():
        vol = df_asset[v].astype(float)
        out[f"{asset_name}__logvol"]  = np.log1p(vol)
        out[f"{asset_name}__vol_chg"] = vol.pct_change()

    return out.replace([np.inf, -np.inf], np.nan)


# ============================================================
# 3) Build tensors for LSTM (NO leakage, COVID safe)
# ============================================================
def make_features_and_labels_from_df(
    features: pd.DataFrame,
    prices: pd.DataFrame,
    lookback=50,
    shift_features_by_1=True,
    include_price_ret_blocks=True
):
    prices = prices.sort_index()
    rets = prices.pct_change()

    idx = features.index.intersection(prices.index).intersection(rets.index)
    features = features.loc[idx]
    prices   = prices.loc[idx]
    rets     = rets.loc[idx]

    # anti-leakage
    if shift_features_by_1:
        features = features.shift(1)
        prices   = prices.shift(1)
        rets     = rets.shift(1)

    # X = [prices | returns | static features]
    if include_price_ret_blocks:
        Xdf = pd.concat([prices, rets, features], axis=1)
    else:
        Xdf = features

    # target: r_t 
    y_rets = prices.pct_change().loc[idx]

    df_all = pd.concat([Xdf, y_rets], axis=1)

    # drop ONLY if target NaN, not for features
    df_all = df_all.dropna(subset=y_rets.columns)
    df_all = df_all.fillna(0.0)

    F = Xdf.shape[1]
    A = y_rets.shape[1]

    Xmat = df_all.iloc[:, :F].values
    Ymat = df_all.iloc[:, F:F+A].values
    dates = df_all.index.to_numpy()

    X, Y, D = [], [], []
    for t in range(lookback, len(df_all)):
        X.append(Xmat[t - lookback:t, :])
        Y.append(Ymat[t, :])
        D.append(dates[t])

    return np.asarray(X, np.float32), np.asarray(Y, np.float32), pd.to_datetime(D), A


# ============================================================
# 4) RUN ALL
# ============================================================
asset_files = {
    "VTI": "VTI 2006-2020.csv",
    "AGG": "AGG 2006-2020.csv",
    "DBC": "DBC 2006-2020.csv",
    "VIX": "VIX 2006-2020.csv",
}

# load OHLCV
ohlcv = pd.concat(
    [load_asset_csv_full(fp, a) for a, fp in asset_files.items()],
    axis=1
).sort_index()

# static features 
features = pd.concat(
    [build_static_features_from_ohlcv(ohlcv, a) for a in asset_files.keys()],
    axis=1
).sort_index()

# drop columns always NaN (e.g. VIX volume)
features = features.dropna(axis=1, how="all")

# close prices for targets
prices = pd.concat(
    [ohlcv[f"{a}__close"].rename(a) for a in asset_files.keys()],
    axis=1
)

# tensors
X, Y, dates, A = make_features_and_labels_from_df(
    features,
    prices,
    lookback=50,
    shift_features_by_1=True
)

print("OHLCV:", ohlcv.shape)
print("Features (static):", features.shape)
print("X:", X.shape, "Y:", Y.shape, "dates:", dates.min(), "→", dates.max(), "A:", A)

print("NaN in features:", features.isna().sum().sum())
print("NaN in X:", np.isnan(X).sum())
print("NaN in Y:", np.isnan(Y).sum())

OHLCV: (3806, 20)
Features (static): (3806, 12)
X: (3701, 50, 20) Y: (3701, 4) dates: 2006-04-21 00:00:00 → 2020-12-31 00:00:00 A: 4
NaN in features: 207
NaN in X: 0
NaN in Y: 0


/var/folders/jq/5vw75gy11gn75l0yyz2h7fs80000gn/T/ipykernel_55487/1244501497.py:67: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change()
/var/folders/jq/5vw75gy11gn75l0yyz2h7fs80000gn/T/ipykernel_55487/1244501497.py:87: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  y_rets = prices.pct_change().loc[idx]


### Updated pipeline 
Compared to the initial replication pipeline, we update the training procedure to explicitly incorporate cross-validation using a 10% temporal hold-out of the training set. For each expanding training window, the most recent 10% of observations is reserved for validation, while the remaining data are used for model fitting.

To reduce the computational cost of hyperparameter selection, cross-validation is performed using a reduced number of training epochs and full-batch optimization. 

In [ ]:
def modified_return_eq7_sequence_impl(x, r_t, w_tm1, n_assets, sigma_tgt, C=1e-4, eps=1e-8, use_scaling=True):
    r_win = x[:, :, n_assets:2*n_assets]
    sigma_tm1 = ewma_sigma_from_returns_window(r_win, span=50, eps=eps)

    w_tm2 = tf.concat([w_tm1[:1], w_tm1[:-1]], axis=0)
    sigma_tm2 = tf.concat([sigma_tm1[:1], sigma_tm1[:-1]], axis=0)

    if use_scaling:
        s_tgt = tf.cast(sigma_tgt, tf.float32)
        scale_tm1 = s_tgt / (sigma_tm1 + eps)
        scale_tm2 = s_tgt / (sigma_tm2 + eps)
        wS_tm1 = scale_tm1 * w_tm1
        wS_tm2 = scale_tm2 * w_tm2
    else:
        wS_tm1 = w_tm1
        wS_tm2 = w_tm2

    gross = tf.reduce_sum(wS_tm1 * r_t, axis=1)
    turnover = tf.reduce_sum(tf.abs(wS_tm1 - wS_tm2), axis=1)
    cost = tf.cast(C, tf.float32) * turnover
    return gross - cost, turnover, cost


@tf.function(
    reduce_retracing=True,
    input_signature=[
        tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),  # x
        tf.TensorSpec(shape=(None, None), dtype=tf.float32),        # r_t
        tf.TensorSpec(shape=(None, None), dtype=tf.float32),        # w_tm1
        tf.TensorSpec(shape=(), dtype=tf.int32),                    # n_assets
        tf.TensorSpec(shape=(), dtype=tf.float32),                  # sigma_tgt
        tf.TensorSpec(shape=(), dtype=tf.float32),                  # C
        tf.TensorSpec(shape=(), dtype=tf.float32),                  # eps 
        tf.TensorSpec(shape=(), dtype=tf.bool),                     # use_scaling
    ],
)
def modified_return_eq7_sequence_tf(x, r_t, w_tm1, n_assets, sigma_tgt, C, eps, use_scaling):
    return modified_return_eq7_sequence_impl(
        x, r_t, w_tm1,
        n_assets=n_assets,
        sigma_tgt=sigma_tgt,
        C=C,
        eps=eps,
        use_scaling=use_scaling
    )

In [ ]:
class LSTMPortfolioEq7(tf.keras.Model):
    def __init__(self, n_assets, lstm_units=64, sigma_tgt=0.10, C=1e-4,
                 use_scaling=True, train_on_eq7=True, eps=1e-8):
        super().__init__()
        self.n_assets = int(n_assets)
        self.sigma_tgt = float(sigma_tgt)
        self.C = float(C)
        self.use_scaling = bool(use_scaling)
        self.train_on_eq7 = bool(train_on_eq7)
        self.eps = float(eps)

        self.lstm = tf.keras.layers.LSTM(lstm_units)
        self.out  = tf.keras.layers.Dense(self.n_assets, activation="softmax")

    def call(self, x, training=False):
        h = self.lstm(x, training=training)
        return self.out(h)

    def _eq7(self, x, r_t, w_tm1):
        # no retracing 
        return modified_return_eq7_sequence_tf(
            tf.cast(x, tf.float32),
            tf.cast(r_t, tf.float32),
            tf.cast(w_tm1, tf.float32),
            tf.constant(self.n_assets, tf.int32),
            tf.constant(self.sigma_tgt, tf.float32),
            tf.constant(self.C, tf.float32),
            tf.constant(self.eps, tf.float32),
            tf.constant(self.use_scaling, tf.bool),
        )

    def train_step(self, data):
        x, r_t = data
        with tf.GradientTape() as tape:
            w_tm1 = self(x, training=True)

            if self.train_on_eq7:
                Rp, turnover, cost = self._eq7(x, r_t, w_tm1)
            else:
                # GROSS return: no scaling, no costs
                Rp = tf.reduce_sum(w_tm1 * tf.cast(r_t, tf.float32), axis=1)  # (batch,)
                turnover = tf.zeros_like(Rp)
                cost = tf.zeros_like(Rp)

            J = sharpe_ratio(Rp)
            loss = -J

        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {
            "loss": loss,
            "sharpe_obj": J,
            "avg_turnover": tf.reduce_mean(turnover),
            "avg_cost": tf.reduce_mean(cost),
        }

    def test_step(self, data):
        x, r_t = data
        w_tm1 = self(x, training=False)
        Rp_mod, turnover, cost = self._eq7(x, r_t, w_tm1)

        J = sharpe_ratio(Rp_mod)
        loss = -J
        return {
            "loss": loss,
            "sharpe_obj": J,
            "avg_turnover": tf.reduce_mean(turnover),
            "avg_cost": tf.reduce_mean(cost),
        }

In [ ]:
def fit_on_period_with_val(
    X_train_full, Y_train_full, n_assets,
    lstm_units=64, sigma_tgt=0.10, C=1e-4,
    lr=1e-3, epochs=100, val_frac=0.10,
    use_early_stopping=False,  # PAPER: "stops after 100 epochs" → default False
    use_scaling=True,
    train_on_eq7=True,
):
    N = len(X_train_full)
    n_val = max(1, int(np.floor(val_frac * N)))
    if N - n_val < 50:
        raise ValueError(f"ning too short after split val: N={N}, n_val={n_val}")

    # temporal split: train = initial part, val = last 10%
    X_tr, Y_tr = X_train_full[:-n_val], Y_train_full[:-n_val]
    X_va, Y_va = X_train_full[-n_val:], Y_train_full[-n_val:]

    model = LSTMPortfolioEq7(
        n_assets=n_assets,
        lstm_units=lstm_units,
        sigma_tgt=sigma_tgt,
        C=C,
        use_scaling=use_scaling,
        train_on_eq7=train_on_eq7
    )
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), run_eagerly=False)

    callbacks = []
    if use_early_stopping:
        callbacks.append(tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            mode="min",
            patience=10,
            min_delta=1e-6,
            restore_best_weights=True
        ))

    
    hist = model.fit(
        X_tr, Y_tr,
        validation_data=(X_va, Y_va),
        epochs=epochs,
        batch_size=len(X_tr),   
        shuffle=False,          # time ordered
        verbose=0,
        callbacks=callbacks
    )
    return model, hist

In [ ]:
def tune_hparams_on_period_fast(
    X_train_full, Y_train_full, n_assets,
    units_grid=(32, 64, 128),
    lr_grid=(1e-4, 3e-4, 1e-3),
    epochs_tune=30,               # few epochs to choose hyperparams
    val_frac=0.10,
    use_early_stopping=True,      # recommended in tuning
    use_scaling=True,
    train_on_eq7=True,
    sigma_tgt=0.10,
    C=1e-4,
    verbose=0
):
    """
    Time-series CV (last val_frac as validation).
    FULL-BATCH training.
    Score = max validation Sharpe (equivalente a min val_loss).
    """
    results = []

    for units in units_grid:
        for lr in lr_grid:
            tf.keras.backend.clear_session()

            model, hist = fit_on_period_with_val(
                X_train_full, Y_train_full,
                n_assets=n_assets,
                lstm_units=int(units),
                sigma_tgt=sigma_tgt,
                C=C,
                lr=float(lr),
                epochs=epochs_tune,
                val_frac=val_frac,
                use_early_stopping=use_early_stopping,
                use_scaling=use_scaling,
                train_on_eq7=train_on_eq7,
            )

            h = hist.history

            
            best_val_loss = float(np.min(h["val_loss"]))
            best_val_sharpe = -best_val_loss

            results.append({
                "lstm_units": int(units),
                "lr": float(lr),
                "best_val_loss": best_val_loss,
                "best_val_sharpe": best_val_sharpe,
            })

            if verbose:
                print(
                    f"units={units:>3}, lr={lr:.1e} | "
                    f"val_sharpe={best_val_sharpe:.4f}"
                )

    results_df = (
        pd.DataFrame(results)
        .sort_values("best_val_sharpe", ascending=False)
        .reset_index(drop=True)
    )

    best = results_df.iloc[0]
    best_params = {
        "lstm_units": int(best["lstm_units"]),
        "lr": float(best["lr"]),
    }

    return best_params, results_df

def refit_best_model_on_period(
    X_train_full, Y_train_full, n_assets,
    best_params,
    sigma_tgt=0.10,
    C=1e-4,
    lr=None,                      # if None, use best_params["lr"]
    epochs_refit=100,             # long final training 
    val_frac=0.10,
    use_early_stopping=False,     
    use_scaling=True,
    train_on_eq7=True,
):
    lr_used = float(best_params["lr"] if lr is None else lr)

    return fit_on_period_with_val(
        X_train_full, Y_train_full,
        n_assets=n_assets,
        lstm_units=int(best_params["lstm_units"]),
        sigma_tgt=sigma_tgt,
        C=C,
        lr=lr_used,
        epochs=epochs_refit,
        val_frac=val_frac,
        use_early_stopping=use_early_stopping,
        use_scaling=use_scaling,
        train_on_eq7=train_on_eq7,
    )

In [18]:
def backtest_period(model, X_period, Y_period, n_assets, sigma_tgt=0.10, C=1e-4, use_scaling=True, eps=1e-8):
    x = tf.convert_to_tensor(X_period, tf.float32)
    r = tf.convert_to_tensor(Y_period, tf.float32)

    w = model(x, training=False)

    Rp_mod, turnover, cost = modified_return_eq7_sequence_tf(
        x, r, w,
        tf.constant(int(n_assets), tf.int32),
        tf.constant(float(sigma_tgt), tf.float32),
        tf.constant(float(C), tf.float32),
        tf.constant(float(eps), tf.float32),
        tf.constant(bool(use_scaling), tf.bool),
    )

    return w.numpy(), Rp_mod.numpy(), turnover.numpy(), cost.numpy()

In [19]:
def run_biennial_retraining_with_val(
    X, Y, dates, A,
    sigma_tgt=0.10,
    C_train=1e-4,
    C_eval=1e-4,
    epochs_tune=30,
    epochs_refit=100,
    val_frac=0.10,
    train_start="2006-01-01",
    first_train_end="2010-12-31",
    test_start="2011-01-01",
    test_end="2020-04-30",
    step_years=2,
    use_scaling=True,
    train_on_eq7=True,
    units_grid=(32, 64),
    lr_grid=(1e-3, 3e-4, 1e-4),
):
    """
    Expanding-window retraining every `step_years`.
    Hyperparameters selected via time-series CV (last `val_frac`).
    Final model refit on full training window.
    """

    # --- date parsing ---
    train_start = pd.to_datetime(train_start)
    test_start  = pd.to_datetime(test_start)
    test_end    = pd.to_datetime(test_end)
    train_end   = pd.to_datetime(first_train_end)

    # --- training cost handling ---
    if train_on_eq7:
        C_train_used = float(C_train)
    else:
        C_train_used = 0.0

    all_weights, all_Rp, all_turnover, all_cost, all_dates = [], [], [], [], []

    # log CV info per blocco
    cv_logs = []

    current_test_start = test_start

    # --- biennial loop ---
    while current_test_start <= test_end:

        current_test_end = min(
            current_test_start + pd.DateOffset(years=step_years) - pd.DateOffset(days=1),
            test_end
        )

        # ---------- TRAIN (expanding) ----------
        tr_idx = idx_between(dates, train_start, train_end)
        if len(tr_idx) < 100:
            raise ValueError(
                f"Pochi dati di training tra {train_start.date()} e {train_end.date()}: {len(tr_idx)}"
            )

        # --- 1) hyperparameter tuning ---
        best_params, cv_table = tune_hparams_on_period_fast(
            X[tr_idx], Y[tr_idx],
            n_assets=A,
            units_grid=units_grid,
            lr_grid=lr_grid,
            epochs_tune=epochs_tune,
            val_frac=val_frac,
            use_scaling=use_scaling,
            train_on_eq7=train_on_eq7,
            sigma_tgt=sigma_tgt,
            C=C_train_used,
        )

        # --- 2) refit best model on full training window ---
        model, hist = refit_best_model_on_period(
            X[tr_idx], Y[tr_idx],
            n_assets=A,
            best_params=best_params,
            sigma_tgt=sigma_tgt,
            C=C_train_used,
            epochs_refit=epochs_refit,
            val_frac=val_frac,
            use_scaling=use_scaling,
            train_on_eq7=train_on_eq7,
        )

        # ---------- TEST (frozen parameters) ----------
        te_idx = idx_between(dates, current_test_start, current_test_end)
        if len(te_idx) == 0:
            break

        w, Rp_mod, turnover, cost = backtest_period(
            model,
            X[te_idx],
            Y[te_idx],
            n_assets=A,
            sigma_tgt=sigma_tgt,
            C=C_eval,
            use_scaling=use_scaling,
        )

        all_weights.append(w)
        all_Rp.append(Rp_mod)
        all_turnover.append(turnover)
        all_cost.append(cost)
        all_dates.append(dates[te_idx])

        # --- log CV info ---
        cv_logs.append({
            "train_end": train_end,
            "test_start": current_test_start,
            "test_end": current_test_end,
            "best_params": best_params,
            "cv_table": cv_table,
        })

        # update expanding window
        train_end = current_test_end
        current_test_start = current_test_end + pd.DateOffset(days=1)

    # ---------- aggregate results ----------
    W  = np.vstack(all_weights)
    Rp = np.concatenate(all_Rp)
    TO = np.concatenate(all_turnover)
    CO = np.concatenate(all_cost)
    D  = pd.to_datetime(np.concatenate(all_dates))

    out = (
        pd.DataFrame(
            {"Rp_mod": Rp, "turnover": TO, "cost": CO},
            index=D
        )
        .sort_index()
    )
    out["cum_return"] = (1.0 + out["Rp_mod"]).cumprod()

    # ---------- metadata (fondamentale) ----------
    out.attrs["train_on_eq7"]   = bool(train_on_eq7)
    out.attrs["use_scaling"]    = bool(use_scaling)
    out.attrs["sigma_tgt"]      = float(sigma_tgt)
    out.attrs["C_train_used"]   = float(C_train_used)
    out.attrs["C_eval"]         = float(C_eval)
    out.attrs["step_years"]     = int(step_years)
    out.attrs["train_start"]    = str(train_start.date())
    out.attrs["first_train_end"]= str(first_train_end)
    out.attrs["test_start"]     = str(test_start.date())
    out.attrs["test_end"]       = str(test_end.date())

    # CV logs (per blocco)
    out.attrs["cv_logs"] = cv_logs

    return W, out

In [ ]:
TRADING_DAYS = 252

def max_drawdown_from_returns(r: pd.Series) -> float:
    """MDD su cumulata (1+r)."""
    cum = (1.0 + r).cumprod()
    dd = cum / cum.cummax() - 1.0
    return float(dd.min())  # negativo

def paper_metrics_from_returns(r: pd.Series, periods_per_year=TRADING_DAYS) -> dict:
    r = r.dropna().astype(float)
    if len(r) < 10:
        raise ValueError("Serie troppo corta per metriche robuste.")

    mu_d = r.mean()
    sd_d = r.std(ddof=1)

    
    E_R   = mu_d * periods_per_year
    Std_R = sd_d * np.sqrt(periods_per_year)

    sharpe = (mu_d / (sd_d + 1e-12)) * np.sqrt(periods_per_year)

    # downside deviation: 
    neg = r[r < 0]
    if len(neg) > 1:
        dd_d = neg.std(ddof=1)
    else:
        dd_d = 0.0
    DD_R = dd_d * np.sqrt(periods_per_year)

    sortino = (mu_d / (dd_d + 1e-12)) * np.sqrt(periods_per_year) if dd_d > 0 else np.nan

    mdd = max_drawdown_from_returns(r)

    pct_pos = float((r > 0).mean())

    pos = r[r > 0]
    ave_p = pos.mean() if len(pos) > 0 else np.nan
    ave_l = neg.mean() if len(neg) > 0 else np.nan  
    ave_p_over_ave_l = float(ave_p / (abs(ave_l) + 1e-12)) if (len(pos) > 0 and len(neg) > 0) else np.nan

    return {
        "E(R) (ann.)": E_R,
        "Std(R) (ann.)": Std_R,
        "Sharpe (ann.)": sharpe,
        "DD(R) (ann.)": DD_R,
        "Sortino (ann.)": sortino,
        "MDD": mdd,
        "% of + Ret": pct_pos,
        "Ave. P / Ave. L": ave_p_over_ave_l,
    }

def backtest_report(perf: pd.DataFrame, periods_per_year=TRADING_DAYS,
                    by_blocks: bool = True) -> pd.DataFrame:
   
    if "Rp_mod" not in perf.columns:
        raise ValueError("perf must contain the column 'Rp_mod'.")

    perf = perf.sort_index()

    rows = []
    m_all = paper_metrics_from_returns(perf["Rp_mod"], periods_per_year=periods_per_year)
    if "turnover" in perf.columns:
        m_all["Avg Turnover"] = float(perf["turnover"].mean())
    if "cost" in perf.columns:
        m_all["Avg Cost"] = float(perf["cost"].mean())
    m_all["Start"] = perf.index.min()
    m_all["End"]   = perf.index.max()
    m_all["N"]     = int(perf["Rp_mod"].dropna().shape[0])
    rows.append(("ALL", m_all))

    
    if by_blocks:
        start_year = int(perf.index.min().year)
        end_year   = int(perf.index.max().year)

        
        y = start_year
        while y <= end_year:
            block_start = pd.Timestamp(year=y, month=1, day=1)
            block_end   = pd.Timestamp(year=min(y+1, end_year), month=12, day=31)

            block = perf.loc[(perf.index >= block_start) & (perf.index <= block_end)]
            if len(block) >= 10:
                m = paper_metrics_from_returns(block["Rp_mod"], periods_per_year=periods_per_year)
                if "turnover" in block.columns:
                    m["Avg Turnover"] = float(block["turnover"].mean())
                if "cost" in block.columns:
                    m["Avg Cost"] = float(block["cost"].mean())
                m["Start"] = block.index.min()
                m["End"]   = block.index.max()
                m["N"]     = int(block["Rp_mod"].dropna().shape[0])
                rows.append((f"{y}-{y+1}", m))
            y += 2

    report = pd.DataFrame({name: vals for name, vals in rows}).T

    
    ordered_cols = [
        "E(R) (ann.)","Std(R) (ann.)","Sharpe (ann.)","DD(R) (ann.)","Sortino (ann.)",
        "MDD","% of + Ret","Ave. P / Ave. L",
        "Avg Turnover","Avg Cost","N","Start","End"
    ]
    existing = [c for c in ordered_cols if c in report.columns]
    report = report[existing]
    return report



In [21]:
modified_return_eq7_sequence = modified_return_eq7_sequence_tf

### Final evaluation
We recover the same network architecture proposed in the paper and observe that extensive feature engineering may introduce additional noise. Conversely, raw prices and returns appear to constitute a more appropriate input representation for the LSTM in this setting.

In [ ]:
SIGMA_DAILY_10 = 0.10 / np.sqrt(252.0)

scenarios = [
    ("No scaling, C=0",         False, SIGMA_DAILY_10, 0.0),
    ("Scaling 10% ann, C=1bp",  True,  SIGMA_DAILY_10, 1e-4),
    ("Scaling 10% ann, C=10bp", True,  SIGMA_DAILY_10, 1e-3),
]

all_reports = []
all_cv_logs = []  

for name, use_scaling, sigma_tgt, C in scenarios:
    tf.keras.backend.clear_session()

    W, perf = run_biennial_retraining_with_val(
        X, Y, dates, A,
        sigma_tgt=sigma_tgt,
        C_train=1e-4,            
        C_eval=C,                
        val_frac=0.10,
        use_scaling=use_scaling,
        train_on_eq7=False,     
        # ---- CV settings ----
        units_grid=(32, 64),
        lr_grid=(1e-3, 3e-4, 1e-4),
        epochs_tune=30,
        epochs_refit=100,
    )

    # ---- report performance ----
    rep = backtest_report(perf, by_blocks=False)
    rep.insert(0, "scenario", name)
    rep.insert(1, "use_scaling", use_scaling)
    rep.insert(2, "sigma_tgt_daily", sigma_tgt)
    rep.insert(3, "C_eval", C)

        # ---- CV settings ----
    rep["cv_used"] = True
    rep["cv_units_grid"] = "32,64"
    rep["cv_lr_grid"] = "1e-3,3e-4,1e-4"

    # ---- hyperparams chosen ----
    last_cv = perf.attrs["cv_logs"][-1]
    rep["cv_best_lstm_units"] = last_cv["best_params"]["lstm_units"]
    rep["cv_best_lr"] = last_cv["best_params"]["lr"]

    all_reports.append(rep)

   
    all_cv_logs.append({
        "scenario": name,
        "cv_logs": perf.attrs["cv_logs"]
    })

summary = pd.concat(all_reports, axis=0).reset_index(drop=True)
summary

2025-12-30 22:26:20.703812: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2025-12-30 22:26:20.704474: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

,scenario,use_scaling,sigma_tgt_daily,C_eval,E(R) (ann.),Std(R) (ann.),Sharpe (ann.),DD(R) (ann.),Sortino (ann.),MDD,...,Avg Turnover,Avg Cost,N,Start,End,cv_used,cv_units_grid,cv_lr_grid,cv_best_lstm_units,cv_best_lr
0,"No scaling, C=0",False,0.006299,0.0000,0.378584,0.236858,1.598361,0.122303,3.095467,-0.239376,...,0.041004,0.0,2347,2011-01-03 00:00:00,2020-04-30 00:00:00,True,"32,64","1e-3,3e-4,1e-4",64,0.001
1,"Scaling 10% ann, C=1bp",True,0.006299,0.0001,0.033792,0.056616,0.596867,0.043973,0.768473,-0.209268,...,0.045862,0.000005,2347,2011-01-03 00:00:00,2020-04-30 00:00:00,True,"32,64","1e-3,3e-4,1e-4",64,0.001
2,"Scaling 10% ann, C=10bp",True,0.006299,0.0010,0.023408,0.062036,0.377323,0.048073,0.486919,-0.169867,...,0.037116,0.000037,2347,2011-01-03 00:00:00,2020-04-30 00:00:00,True,"32,64","1e-3,3e-4,1e-4",64,0.001
